In [1]:
# Exercise 2

import numpy as np
import torch
from torch import optim
import torch.nn as nn

In [2]:
import csv
wine_path = "../data/p1ch4/tabular-wine/winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",
                         skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [32]:
# 注意此时只进行回归，但是这个数据实际上是分类

wineq = torch.from_numpy(wineq_numpy)
wineq.shape

torch.Size([4898, 12])

In [33]:
# 先划分label
data = wineq[:,:-1]
labels = wineq[:,-1]
print(data.shape, labels.shape)

torch.Size([4898, 11]) torch.Size([4898])


In [7]:
target = labels.unsqueeze(-1)
target,target.shape

(tensor([[6.],
         [6.],
         [6.],
         ...,
         [6.],
         [7.],
         [6.]]),
 torch.Size([4898, 1]))

In [10]:
# 划分训练集和测试集

n_samples = data.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices.shape, val_indices.shape

(torch.Size([3919]), torch.Size([979]))

In [40]:
data_norm = data

# print(torch.mean(data_norm,dim=0))
# print(data_norm[:,0].mean())

mean_data = torch.mean(data_norm, dim=0)
std_data = torch.std(data_norm, dim=0)

print('mean:',mean_data, mean_data.shape)
print('std:',std_data,std_data.shape)

data_norm = (data_norm - mean_data)/std_data

print('\n')
print('after normalization:\n')
print(torch.mean(data_norm,dim=0))
# print(data_norm[:,0].mean(),data_norm[:,1].mean())
print(torch.std(data_norm,dim=0))

train_data = data_norm[train_indices]
print(train_data.shape)
train_label = target[train_indices]

val_data = data_norm[val_indices]
val_label = target[val_indices]


mean: tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01]) torch.Size([11])
std: tensor([8.4387e-01, 1.0079e-01, 1.2102e-01, 5.0721e+00, 2.1848e-02, 1.7007e+01,
        4.2498e+01, 2.9909e-03, 1.5100e-01, 1.1413e-01, 1.2306e+00]) torch.Size([11])


after normalization:

tensor([-1.8147e-07,  2.1612e-07,  6.4643e-08,  4.1278e-08,  4.9456e-08,
        -8.7229e-08,  2.0250e-08, -5.4518e-08,  8.3471e-07,  4.3498e-07,
         4.9456e-08])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000])
torch.Size([3919, 11])


In [29]:
a = torch.tensor([[1,2],[3,4]])
print(a)
b = torch.tensor([1,3])
s = torch.tensor([2,1])
print(a-b)
(a-b)/s

tensor([[1, 2],
        [3, 4]])
tensor([[ 0, -1],
        [ 2,  1]])


tensor([[ 0., -1.],
        [ 1.,  1.]])

In [41]:
# model

neuron_count = 16
input_shape = train_data.shape[1]

seq_model = nn.Sequential(
            nn.Linear(input_shape, neuron_count), # <1>
            nn.Tanh(),
            nn.Linear(neuron_count, 1)) # <2>
seq_model

Sequential(
  (0): Linear(in_features=11, out_features=16, bias=True)
  (1): Tanh()
  (2): Linear(in_features=16, out_features=1, bias=True)
)

In [42]:
# loss
# nn.MSELoss()

In [47]:
# opimizer
# SGD
optimizer = optim.Adam(seq_model.parameters(), lr=1e-3)

In [49]:
# train
def training_loop(n_epochs, optimizer, model, loss_fn, data_train, label_train,
                  data_test, label_test):
    for epoch in range(1, n_epochs + 1):
        train_predict = model(data_train) # <1>
        loss_train = loss_fn(train_predict, label_train)

        test_predict = model(data_test) # <1>
        loss_val = loss_fn(test_predict, label_test)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [51]:
training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    data_train = train_data,
    label_train = train_label, 
    data_test = val_data,
    label_test = val_label)
    
# print('output', seq_model(val_data)) # 预测值
# print('answer', val_label) # 实际值
# print('hidden', seq_model.hidden_linear.weight.grad)

Epoch 1, Training loss 0.4742, Validation loss 0.5239
Epoch 1000, Training loss 0.4688, Validation loss 0.5234
Epoch 2000, Training loss 0.4627, Validation loss 0.5260
Epoch 3000, Training loss 0.4577, Validation loss 0.5251
Epoch 4000, Training loss 0.4516, Validation loss 0.5221
Epoch 5000, Training loss 0.4458, Validation loss 0.5264
